In [1]:
import os
from feon.sa import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from feon.tools import pair_wise
from scipy.special import ellipk
from matplotlib import cm
import imageio.v2 as imageio

In [22]:
#The difference between the rough and exact solutions
def delta_T(length = 1,g = 9.812):
    fig,ax1 = plt.subplots(figsize = (8,6))
    plt.rcParams['font.family'] = 'Times New Roman'
    x = np.linspace(0,90,1800)
    y1 = 2 * np.pi * np.sqrt(length / g)
    y2 = 4 * np.sqrt(length / g) * ellipk(np.power(np.sin(x / 180 * np.pi / 2),2)) / y1
    y1 = np.array([y1 for i in range(len(x))]) / y1
    delta = (y2 - y1) / y2 * 100
    ax1.plot(x,y1,color = '#ECE70F',label = 'Rough solution')
    ax1.plot(x,y2,color = '#00A14E',label = 'Exact solution')
    ax1.plot([-1,-1],[-1,-1],linestyle = '--',color = '#688CCB',label = 'Relative error')
    ax1.tick_params(axis = 'x',labelsize = 14)
    ax1.tick_params(axis = 'y',labelsize = 14)
    ax1.set_xlim(0,90)
    ax1.set_ylim(0.95,1.25)
    ax1.set_xlabel('Angle (°)',fontsize = 16)
    ax1.set_ylabel('Multiples of the rough solution',fontsize = 16)
    ax1.legend(fontsize = 16,loc = 'upper left')
    ax2 = ax1.twinx()
    ax2.set_ylim(0,18)
    ax2.tick_params(axis = 'y',labelsize = 14)
    ax2.plot(x,delta,linestyle = '--',color = '#688CCB')
    ax2.set_ylabel('Relative error (%)',fontsize = 16)
    plt.savefig('T.png',bbox_inches = 'tight',dpi = 1200)
    plt.show()

In [3]:
#Pendulum equation
def pendulum_equations(solution,time,length,g = 9.812):
    theta,v = solution
    d_theta = v
    d_v  = - g / length * np.sin(theta)
    return d_theta,d_v

In [4]:
#Solve for the force
def force(length = 0.93632,init_angle = 60,time = 'auto',delta_time = 0.01,m = 64.3,g = 9.812,draw_f = False):
    periods = 4 * np.sqrt(length / g) * ellipk(np.power(np.sin(init_angle / 180 * np.pi / 2),2))
    if time == 'auto':
        time = periods
    time = np.arange(0,time,delta_time)
    if abs(time[-1] - periods) > 1e-3:
        time = np.append(time,periods)
    data = odeint(pendulum_equations,(init_angle / 180 * np.pi,0),time,args = (length,g)).T
    theta,w = data[0],data[1]
    f = (m * g * np.cos(theta) + m * np.power(w,2) * length) * 1e-3
    fz = f * np.cos(theta)
    fx = f * np.sin(theta)
    if draw_f == True:
        print('T = ' + str(periods))
        rough_periods = 2 * np.pi * np.sqrt(length / g)
        print('Rough_T = ' + str(rough_periods))
        print('Relative error = ' + str((periods - rough_periods) / periods))
        fig,ax = plt.subplots(figsize = (8,6))
        plt.rcParams['font.family'] = 'Times New Roman'
        plt.plot(time,f * 1e3,color = '#EF767B',label = 'F')
        plt.plot(time,fx * 1e3,color = '#43A3EF',label = '${F_{horizontal}}$')
        plt.plot(time,fz * 1e3,color = '#FF7F00',label = '${F_{vertical}}$')
        plt.xlabel('Time (s)',fontsize = 16)
        plt.ylabel('F (N)',fontsize = 16)
        plt.xticks(fontsize = 14)
        plt.yticks(fontsize = 14)
        plt.xlim(0,periods)
        plt.ylim(-1000,1500)
        plt.legend(fontsize = 16,loc = 'lower center',ncol = 3)
        plt.savefig('F.png',bbox_inches = 'tight',dpi = 1200)
        plt.show()
    return f,fx,fz,time

In [5]:
#Model building
def build(fx,fz,all_result,delta_x = 0.05,height = 2.55,shoulder_width = 0.48,bar = 2.8,A = np.pi * (0.5 * 28 * 0.001) ** 2,E = 206 * 1e6,G = 79.4 * 1e6,I = np.array([np.pi * (28 * 0.001) ** 4 / 64,np.pi * (28 * 0.001) ** 4 / 64,np.pi * (28 * 0.001) ** 4 / 64])):
    nodes_1 = [Node(0,0,i * delta_x) for i in range(int(round(height / delta_x)))]
    nodes_2 = [Node(0,i * delta_x,height) for i in range(int(round(bar / delta_x)))]
    nodes_3 = [Node(0,bar,height - i * delta_x) for i in range(int(round(height / delta_x)) + 1)]
    nodes = nodes_1 + nodes_2 + nodes_3
    elements = []
    for nd in pair_wise(nodes):
        elements.append(Beam3D11(nd,E,G,A,I))
    node1 = len(nodes_1) + int((bar - shoulder_width) / 2 / delta_x)
    node2 = len(nodes_1) + int((bar + shoulder_width) / 2 / delta_x)
    s = System()
    s.add_nodes(nodes)
    s.add_elements(elements)
    s.add_fixed_sup(0)
    s.add_fixed_sup(len(nodes) - 1)
    s.add_fixed_sup(len(nodes_1) - 1)
    s.add_fixed_sup(len(nodes_2) + len(nodes_1))
    s.add_node_force(nodes[node1].ID,Fx = fx / 2,Fz = fz / 2)
    s.add_node_force(nodes[node2].ID,Fx = fx / 2,Fz = fz / 2)
    s.solve()
    all_result.append([[i.get_disp()['Ux'],i.get_disp()['Uy'],i.get_disp()['Uz']] for i in nodes])
    return all_result

In [6]:
#Calculation results
def draw_fig(all_result,f,fx,fz,time,height = 2.55,delta_x = 0.05):
    all_ux = []
    all_uz = []
    for i_t in  all_result:
        all_nodes = np.array(i_t[int(round(height / delta_x)):-int(round(height / delta_x)) - 1]).T
        ux = all_nodes[0]
        uz = all_nodes[2]
        all_ux.append(ux)
        all_uz.append(uz)
    min_ux,min_uz,max_ux,max_uz = -0.012,0,0.012,0.024
    for idx,item in enumerate(all_ux):
        fig = plt.figure(figsize = (8,10))
        plt.rcParams['font.family'] = 'Times New Roman'
        gs = fig.add_gridspec(10,1)
        ax1 = fig.add_subplot(gs[:1,0])
        x_strain =  np.zeros((1,len(item)))
        for i in range(len(item)):
            x_strain[0,i] = item[i]
        cax = ax1.imshow(x_strain,interpolation = 'nearest',cmap = cm.coolwarm,vmin = min_ux,vmax = max_ux)
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax1.set_title('${S_{horizontal}}$',fontsize = 16)
        ax2 = fig.add_subplot(gs[1:2,0])
        color_bar1 = np.array([np.linspace(min_ux,max_ux,len(item))])
        cax = ax2.imshow(color_bar1,interpolation = 'nearest',cmap = cm.coolwarm,vmin = min_ux,vmax = max_ux)
        ax2.set_xticks([(len(item) - 1) * ((i + 0.012) / 0.024) for i in [-0.012,-0.008,-0.004,0.000,0.004,0.008,0.012]],[-12,-8,-4,'0',4,8,12])
        ax2.set_xlabel('Displacement (mm)',fontsize = 16)
        ax2.set_yticks([])
        ax2.tick_params(axis = 'x', labelsize = 14)
        ax3 = fig.add_subplot(gs[2:3,0])
        z_strain =  np.zeros((1,len(item)))
        for i in range(len(item)):
            z_strain[0,i] = all_uz[idx][i]
        cax = ax3.imshow(z_strain,interpolation = 'nearest',cmap = cm.coolwarm,vmin = min_uz,vmax = max_uz)
        ax3.set_xticks([])
        ax3.set_yticks([])
        ax3.set_title('${S_{vertical}}$',fontsize = 16)
        ax4 = fig.add_subplot(gs[3:4,0])
        color_bar2 = np.array([np.linspace(min_uz,max_uz,len(item))])
        cax = ax4.imshow(color_bar2,interpolation = 'nearest',cmap = cm.coolwarm,vmin = min_uz,vmax = max_uz)
        ax4.set_xticks([(len(item) - 1) * (i / 0.024) for i in [0,0.004,0.008,0.012,0.016,0.020,0.024]],['0',4,8,12,16,20,24])
        ax4.set_yticks([])
        ax4.set_xlabel('Displacement (mm)',fontsize = 16)
        ax4.tick_params(axis = 'x', labelsize = 14)
        ax5 = fig.add_subplot(gs[4:10,0])
        ax5.set_xlabel('Time (s)',fontsize = 16)
        ax5.set_ylabel('F (N)',fontsize = 16)
        ax5.tick_params(axis = 'x', labelsize = 14)
        ax5.tick_params(axis = 'y', labelsize = 14)
        ax5.set_xlim(0,time[-1])
        ax5.set_ylim(-1000,1500)
        ax5.plot(time,f * 1e3,color = '#EF767B',label = 'F')
        ax5.plot(time,fx * 1e3,color = '#43A3EF',label = '${F_{horizontal}}$')
        ax5.plot(time,fz * 1e3,color = '#FF7F00',label = '${F_{vertical}}$')
        ax5.scatter([time[idx]],[f[idx] * 1e3],color = '#EF767B')
        ax5.scatter([time[idx]],[fx[idx] * 1e3],color = '#43A3EF')
        ax5.scatter([time[idx]],[fz[idx] * 1e3],color = '#FF7F00')
        the_time = str(round(time[idx],6)) + (6 - len(str(round(time[idx],6)).split('.')[-1])) * '0'
        ax5.text(time[-1] / 2,1400,'Time = %ss' % str(the_time),fontsize = 16,va = 'center',ha = 'center')
        ax5.legend(fontsize = 16,loc = 'lower center',ncol = 3)
        plt.tight_layout()
        plt.savefig('all_pic/pic%s.png' % str(idx),bbox_inches = 'tight',dpi = 400)
        plt.show()

In [7]:
#Create gif from images
def create_gif_from_images(folder,duration = 0.01):
    images = []
    for i in range(len(os.listdir(folder))):
        images.append(imageio.imread(folder + '/pic' + str(i) + '.png'))
    imageio.mimsave('pic.gif',images,duration = duration)

In [8]:
#Add text
def text(x,y,color = 'red'):
    for idx,item in enumerate(x):
        plt.text(x[idx],y[idx] + 0.5,y[idx],color = color,fontsize = 14,ha = 'center',va = 'bottom',rotation = 90)

In [9]:
#Draw a bar chart
def draw_bar():
    fig,ax1 = plt.subplots(figsize = (8,6))
    plt.rcParams['font.family'] = 'Times New Roman'
    width = 0.8
    plt.bar([0,5 * width,10 * width,15 * width],[5.9,10.6,9.2,10.5],width = width,label = 'Excellent',color = '#FEA0A0')
    plt.bar([width,6 * width,11 * width,16 * width],[2.1,3.6,2.9,3.0],width = width,label = 'Good',color = '#AD07E3')
    plt.bar([2 * width,7 * width,12 * width,17 * width],[15.2,21.6,16.6,16.9],width = width,label = 'Pass',color = '#55A0FB')
    plt.bar([3 * width,8 * width,13 * width,18 * width],[76.8,64.2,71.2,69.6],width = width,label = 'Fail',color = '#099A63')
    text([0,5 * width,10 * width,15 * width],[5.9,10.6,9.2,10.5],color = '#FEA0A0')
    text([width,6 * width,11 * width,16 * width],[2.1,3.6,2.9,3.0],color = '#AD07E3')
    text([2 * width,7 * width,12 * width,17 * width],[15.2,21.6,16.6,16.9],color = '#55A0FB')
    text([3 * width,8 * width,13 * width,18 * width],[76.8,64.2,71.2,69.6],color = '#099A63')
    plt.ylim(0,100)
    plt.xticks([1.5 * width,6.5 * width,11.5 * width,16.5 * width],['2019','2020','2021','2022'],fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.ylabel('Percentage (%)',fontsize = 16)
    plt.xlabel('Year',fontsize = 16)
    plt.legend(ncol = 4,fontsize = 16)
    plt.savefig('1.png',bbox_inches = 'tight',dpi = 1200)
    plt.show()

In [ ]:
if __name__ == '__main__':
    delta_T()
    f,fx,fz,time = force(draw_f = True)
    f,fx,fz,time = force()
    all_result = []
    for idx,item in enumerate(fx):
        all_result = build(item,fz[idx],all_result)
    draw_fig(all_result,f,fx,fz,time)
    create_gif_from_images('all_pic')
    draw_bar()